In [1]:
# Modules for bead image analysis

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import os
import sys
import cv2
import csv
from statistics import mean, median, stdev
import pandas as pd; print("Pandas version: {0}".format(pd.__version__)); 

# Avoid user warnings from nikon (?)
import warnings; warnings.simplefilter('ignore',UserWarning)
import nd2reader as nd2; print("nd2 reader version: {0}".format(nd2.__version__))

from tqdm import tqdm_notebook,trange
warnings.simplefilter('ignore',DeprecationWarning) # This should be fine if tqdm not updated; it is the tqdm_notebook thats crying


Pandas version: 1.1.4
nd2 reader version: 3.2.3


In [30]:
class ImageStack(object):
    """Summary: Containing functions to (a) process Nikon images - nd2 file format (b) Extracts relevant file images 
    for further image processing. 
    NOTE: The file form of the nd2 file must be only channels and no frames/time/zstacks etc; 
    TODO: A different nd2 stack will mess up the processing and analysis and will need reprocessiong; 
    A scaleable method will be to have a function that converts it to a channel_stack images
    
    Attributes 
    ----------
    bead_nd2_fpath: (str); Its the absolute path to the nd2 file
    mono_channel: (int) : It is meant to be the channel used for extracting the image of the beads; Typically a 
    non-fluorescent bead channel
    metadata: (list) : It is the nikon nd2 raw metadata
    nd2image: (nd2 image class): This is their proprietary class; The relevant extraction here is the image information 
    that is an array (not yet numpy)
    
    Methods
    -------
    * check_nd2_file_form: This is currently critical to ensure that there is only one stack of channels 
    and not a time or xy series. 
    Returns: True if the form is correct
    * metadata: Quick extraction of metadata; Returns metadata and the nd2image
    * channels: Specific call out to the channels
    * frames: Extracts out the frame information 
    * fname : File name (not the path)
    
    """
    
    def __init__(self,bead_nd2_fpath,mono_channel=0):
        self.bead_nd2_fpath = bead_nd2_fpath
        self.mono_channel = mono_channel
        self.metadata, self.nd2image = self.metadata()
        self.check_nd2_file_form()
      
    def check_nd2_file_form(self):
        assert (len(self.metadata['fields_of_view']) == 1 and len(self.metadata['frames']) == 1),"Metadata {0}".format(self.metadata)
        assert ((len(self.metadata['channels']))>1), "Not enough channels in this image stack"
        return True
    
    def metadata(self):
        nd2image = nd2.ND2Reader(self.bead_nd2_fpath)
        metadata = nd2image.metadata
        return metadata, nd2image
    
    def get_mono_image(self):
        return self.nd2image.get_frame(self.mono_channel)
    
    def channels(self):
        return self.metadata['channels']
    
    def frames(self):
        return self.metadata['frames']
   
    def fname(self):
        return(os.path.split(self.bead_nd2_fpath)[1])


In [166]:
class Beads(ImageStack):
    """
    The Beads class actually tracks the intensity of the beads across the different channels. 
    TODO: Ensures that it is done across channels and the ImageStack inherited is of the form
    Summarizing a dataframe to obtain values
    """
    def __init__(self,bead_nd2_fpath,mono_channel=0):
        super().__init__(bead_nd2_fpath) #I don't like this, but there are too many methods that can be passed down
        self.col_headers = self.make_column_header()
        self.mono_channel = mono_channel
    
    def make_column_header(self):
        channels = self.channels()
        channel_dependent_headers = list()
        col_headers = ["Filename", "Frame ID", "Mask ID", "Mask-X0","Mask-Y0","oRadius (pix)",
                       "Circular Area (pix^2)", "Ring Area (pix^2)", "inner Circle Area (pix^2)"]
        
        for channel_name in channels:
            _header_list = [channel_name + '---' + _head_ 
                            for _head_ in ('oCircle Intensity', 'oCircle Mean Intensity', 'oCircle Median Intensity',
                                           'iCircle Intensity', 'iCircle Mean Intensity', 'iCircle Median Intensity',
                                           'ring Intensity', 'ring Mean Intensity', 'ring Median Intensity')]
            channel_dependent_headers.extend(_header_list)
        col_headers.extend(channel_dependent_headers)
        return col_headers
    
    def make_dataframe(self,row_values):
        allbeads_df = pd.DataFrame(row_values,columns=self.col_headers)
        return allbeads_df
    
    def preprocess_image(self,img,preprocessing_params=['median',5]):
        mod_img1 = img.astype(np.float32)
        mod_img2 = (mod_img1/256).astype('uint8') #Need an 8-bit, single-channel, grayscale input
        # Blurring
        if preprocessing_params[0] == 'median': cv2.medianBlur(mod_img2,preprocessing_params[1])
        if preprocessing_params[0] == 'gaussian': cv2.GaussianBlur(mod_img2,preprocessing_params[1])
        return mod_img2

    def filter_beads(self,circles, min_width=100,max_width=170):
        all_circles = list()
        for circle in circles:
            r = int(circle[2])
            if r in range(min_width,max_width):
                all_circles.append(circle)
        return all_circles

    def find_beads(self,**kwargs):
        """
        hough_params = dict(dp=1.5,minDist=250,param1=45,param2=45,minRadius=100,maxRadius=700)
        circles = t.find_beads(mono_image,**hough_params)
        """
        hough_params = kwargs['hough_params']
        _preprocessing_params = kwargs['preprocessing_params'] #I am barely modifying this
        _processed_img = self.preprocess_image(self.get_mono_image(),_preprocessing_params)
        circles = (cv2.HoughCircles(_processed_img,cv2.HOUGH_GRADIENT,
                                    dp = hough_params['dp'],
                                    minDist = hough_params['minDist'],
                                    param1 = hough_params['param1'],
                                    param2 = hough_params['param2'],
                                    minRadius = hough_params['minRadius'],
                                    maxRadius = hough_params['maxRadius']))
        circles = self.filter_beads(circles[0]) #Filter the beads across the expected width (and not too small)
        # A number of circles with each circle in form = (x,y,radius)
        return circles 
    
    def export_data_frame(self,data_frame, export_folder):
        #display(data_frame)
        _input_fname = os.path.split(self.bead_nd2_fpath)[1].split('.')[0]
        export_fpath = os.path.join(export_folder, _input_fname+'.csv')
        print("Saving data to {0}".format(export_fpath))
        data_frame.to_csv(export_fpath, header=True, index=False)
        return export_fpath
    
    def export_masked_image(self,circles,export_folder):
        #Overlaying a ring around the detected mask
        ## Need to convert the mono image to a gray scale and convert to uint8
        _mono_image = self.get_mono_image()
        _mono_data = _mono_image.astype(np.float64) / np.max(_mono_image) #Normalizing from 0-1
        _mono_data = 255 * _mono_data # Now scaled to 255
        _img = _mono_data.astype(np.uint8)
        raw_mono_image = cv2.cvtColor(_img,cv2.COLOR_GRAY2BGR)
        
        ring_image = cv2.cvtColor(raw_mono_image,cv2.COLOR_BGR2RGB)
        
        for circle in circles:
            x0,y0,r = map(int,circle)
            cv2.circle(img=ring_image, center=(x0,y0), radius=r, color=(204,85,0),thickness=30)

        _image_fname = os.path.split(self.bead_nd2_fpath)[1].split('.')[0]
        export_fpath = os.path.join(export_folder,_image_fname)
        cv2.imwrite(export_fpath+'_ring_'+'.png',ring_image)
        cv2.imwrite(export_fpath+'_raw_'+'.png',_img)

        return export_fpath
            
   

In [167]:
class Bead(ImageStack):
    """
    Main purpose is to extract intensity with varying statistics for a single bead 
    """
    def __init__(self,bead_nd2_fpath, circle, mask_id, mono_channel=0):
        super().__init__(bead_nd2_fpath)
        self.circle = list(map(int,circle)) #circle = x,y,r
        self.mono_image = self.nd2image.get_frame(mono_channel)
        self.image_shape = self.mono_image.shape
        self.ring_thickness = int(30)
        self.mask_id = mask_id

    def get_areas(self):
        x0,y0,r0 = self.circle
        delta = self.ring_thickness/2
        outer_circle_area = np.pi * float(r0+delta) * float(r0+delta)
        inner_circle_area = np.pi * float(r0-delta) * float(r0-delta)
        ring_area = 2 * np.pi * float(r0)*self.ring_thickness
        return [outer_circle_area, ring_area, inner_circle_area]
        
    def draw_mask(self,**kwargs):
        empty_img_mask = np.zeros(self.image_shape,dtype=np.uint16)
        cv2.circle(empty_img_mask,kwargs['center'],kwargs['radius'],kwargs['color'],kwargs['thickness'])
        return empty_img_mask
     
    def get_values_under_mask(self,mask_array,idx):
        raw_image = self.nd2image.get_frame(idx)
        _image_masked_ = np.multiply(raw_image,mask_array)
        sum_bead = np.sum(_image_masked_)
        mean_bead = np.nanmean(np.where(_image_masked_!=0, _image_masked_, np.nan))
        median_bead = np.nanmedian(np.where(_image_masked_!=0, _image_masked_, np.nan))
        return sum_bead,mean_bead,median_bead
        
    def extract_intensity(self):
        # The function is simply to do the intensity calculations
        # Initialize
        row_values = list()
        
        x0,y0,r0 = self.circle
        delta = self.ring_thickness/2
        outer_circle_params = dict(center=(x0,y0),radius=int(r0+delta),color=1,thickness=-1)
        inner_circle_params = dict(center=(x0,y0),radius=int(r0-delta),color=1,thickness=-1)
        ring_params = dict(center=(x0,y0),radius=int(r0),color=1,thickness=self.ring_thickness)
        
        # Create bead mask
        outer_circ_mask = self.draw_mask(**outer_circle_params)
        inner_circ_mask = self.draw_mask(**inner_circle_params)
        ring_mask = self.draw_mask(**ring_params)
        
        # Calculate intensity for each channel
        # Looping through the different masks modes
        for circ_mask in (outer_circ_mask,inner_circ_mask, ring_mask):
            # Looping to extract -- Sum, Mean, Median, Ratio
            for  idx,channel_name in enumerate(self.channels()):
                sum_bead, mean_bead, median_bead = self.get_values_under_mask(circ_mask,idx)
                row_values.extend([sum_bead,mean_bead,median_bead])
        return row_values

    def compile_row(self,row_values):
        complete_row_list = list()
        # Basic bead information
        basic_info_row_values = [self.fname(),self.frames()[0],self.mask_id,self.circle[0],self.circle[1],self.circle[2]] 
        complete_row_list.extend(basic_info_row_values)
        area_row_values = np.array(self.get_areas(),dtype=np.float64)
        complete_row_list.extend(area_row_values)
        
        # Intensity across channels
        intensity_row_values = np.array(row_values,dtype=np.float64)
        complete_row_list.extend(intensity_row_values)
        
        return list(complete_row_list)
        
    
    

In [168]:
def get_fpaths(top_level_folder, ends_pattern='.nd2'): #Someday will make a generic pattern in mymodule
    all_fpath = list()
    for root,dirs,files in os.walk(top_level_folder):
        for file in files:
            if file.endswith(ends_pattern): 
                fpath = os.path.join(root,file) 
                all_fpath.append(fpath)
    return all_fpath


def make_folder(level_folder):
    try:
        os.makedirs(level_folder)
    except OSError as error:
        pass
    return level_folder



In [169]:
def analyze_beads_single_fieldofview(nd2_fpath, **kwargs): 
    '''Beads in a single field of view detected and analysed (image stack of channels)
    The goal also is to ensure the class calls are restricted to only this function
    '''
    complete_row_list = list()

    # Processing a single nd2 file to extract bead information
    beadimage = Beads(nd2_fpath)   ## ATTENTION -- Beads class (all beads)
    circles = beadimage.find_beads(**kwargs) #Don't know why I am just rerouting all of kwargs here directly
    export_folder_path = kwargs['export_folder']
    save_status = kwargs['save_status']

    nbr_circles = len(circles)
    tqdm_descriptor = "Analyzing beads {0}: ".format(os.path.split(nd2_fpath)[1])
    tqdm_color_orange = '#cc5500'

    for mask_id in tqdm_notebook(range(nbr_circles),desc=tqdm_descriptor,colour=tqdm_color_orange,position=1, leave=False):
        circle = circles[mask_id]
        bead = Bead(nd2_fpath,circle,mask_id) ## ATTENTION -- Bead (single bead)
        bead_values = bead.extract_intensity() 
        row_list = bead.compile_row(bead_values)
        complete_row_list.append(row_list)
    
    beads_data_frame = beadimage.make_dataframe(complete_row_list) ## ATTENTION - Recalling the Beads class
    _nbr_rows = beads_data_frame.shape[0]
    if _nbr_rows >1: status = True
    else: status = False

    if save_status in ['data','both']:
        data_export_folder = os.path.join(export_folder_path,'data_frame_export')
        data_export_fpath = make_folder(data_export_folder)
        exported_csv_fpath = beadimage.export_data_frame(beads_data_frame,data_export_fpath)
    elif save_status in ['mask','both']:
        mask_export_folder = os.path.join(export_folder_path,'mask_image_export')
        mask_export_fpath = make_folder(mask_export_folder)
        exported_mask_fpath = beadimage.export_masked_image(circles,mask_export_fpath)
    else:
        pass
    
    ## ASSERTIONS -- MAY NEED TO BE SILENCED
    assert (_nbr_rows == len(circles)), "Circles detected != Dataframe rows"
    assert (nbr_circles > 5), "Can you do better in number of circles detected {0} ?".format(nbr_circles)
    
    return True, beads_data_frame
    

In [170]:
def test_nd2file(test_nd2_fpath):
    status, test_data_frame = analyze_beads_single_fieldofview(test_nd2_fpath, **params)
    _nbr_rows = test_data_frame.shape[0]

    return True, test_data_frame
    
def test_nd2folder(test_nd2_folder,**kwargs):
    all_test_nd2_fpaths = get_fpaths(test_nd2_folder,'nd2')
    all_test_nd2_fpaths.sort()
    
    tqdm_descriptor = "Processing folder : "
    tqdm_color_blue = '#0D4F8B'
    
    counter = 0
    for counter in tqdm_notebook(range(len(all_test_nd2_fpaths)), 
                          desc=tqdm_descriptor, colour=tqdm_color_blue, position=0):
        nd2_fpath = all_test_nd2_fpaths[counter]
        status, test_dataframe = test_nd2file(nd2_fpath)
        counter+=1
        
    print("Completed images in folder : {0}".format(test_nd2_folder))
    
    return True, test_nd2_folder

In [171]:
test_file = '201015_TGNH2_Atto495_MEOH_10X_500ms_flds001.nd2'
test_image_folder = '/home2/software_projects/beadAnalysis/temp/test_images/201015_TGNH2_Atto495_MEOH_10X_500ms'
#check_status, test_dataframe = test_nd2file(os.path.join(test_image_folder,test_file))
test_nd2folder(test_image_folder)
#save_status = ['data','mask','both',None]



Completed images in folder : /home2/software_projects/beadAnalysis/temp/test_images/201015_TGNH2_Atto495_MEOH_10X_500ms


(True,
 '/home2/software_projects/beadAnalysis/temp/test_images/201015_TGNH2_Atto495_MEOH_10X_500ms')

In [ ]:
def analyze_beads_all_images(nd2_folder, *args,**kwargs):
    # The main function iterating through the entire folder of nd2 files to extract the relevant bead information for all the files
    all_nd2_fpaths = get_fpaths(nd2_folder,'.nd2')
    all_nd2_fpaths.sort()
    
    tqdm_descriptor = "Processing folder : {0}".format(all_nd2_fpaths[0])
    tqdm_color_blue = '#0D4F8B'
    
    counter = 0
    for fpath_idx in tqdm(range(len(all_nd2_fpaths)), 
                          desc="\t Processing fpath: {0}".format(all_nd2_fpaths[counter]), colour=tqdm_color_blue, position=0):
        nd2_fpath = all_nd2_fpaths[counter]
        analyze_beads_single_fieldofview(nd2_fpath,**kwargs)
        counter+=1
    

In [ ]:
## Default parameters
hough_params = dict(dp=1.5,minDist=250,param1=45,param2=45,minRadius=100,maxRadius=700)
preprocessing_params = ('median',5) #Someday going or will need to make it better
save_status = 'both' #save_status = ['data','mask','both','None']
export_folder = os.path.join(os.getcwd(),)''

head_folder = os.path.split(test_nd2_fpath)[0]
# Only because it is test; Else should be sent as key:value in params
export_folder_path = os.path.join(head_folder,'test_data_export')
params = dict(hough_params=hough_params, preprocessing_params=preprocessing_params, save_status=save_status,export_folder=export_folder_path)
